## weather 설명
- weather0: 위치 NaN 제거, 경기도 지역 제외, [풍향1, 풍향1(km/h), 풍향10, 풍향10(km/h) NaN값 날려버리기]
- weather1: 일강수, 강수60의 '.'을 0으로 대체, 라벨링을 위해 object type을 float type으로 바꿈
- weather2: 풍속10(km/h)를 풍속10(m/s)로 바꿔버림
- weather3: 풍속1(m/s)을 시간당 평균 내서 풍속 60(m/s)으로 바꿈.

## 문제점
- 풍속1 또는 풍속10일 때는 최대값이 12.61로 나왔는데 풍속60으로 평균치를 내버리니깐 '바람많이'라는 라벨링이 사라져 버리고 최댓값이 4.2
- 이게 과연 좋은 건지 나쁜건지.. 생각해보쟈

-------------------

# 예측 (지도학습)
- target = shared
- feature = hit 수?(유동인구) / 우천(안비, 적은비, 많은비) / 풍속 / 미세먼지 / 각 정거장 별 대여한 고객의 평균 목적지수 / 시간대 (기준 세울 필요. 일단 1시간) / 버스정거장 연계성 지수
- feature 정제해서 만들기
- 예측에 쓰이는 학습 model 공부하기

### 우천 라벨링?
- 맑음 : 0
- 적은비(~시간당 5mm) : 1
- 많은비(>시간당 5mm ) : 2

### 불쾌지수 라벨링? (X)
- [불쾌지수=1.8×기온–0.55×(1–습도)×(1.8×기온–26)+32]
- 불쾌지수 < 70 : 0
- 70 <= 불쾌지수 < 80: 1
- 불쾌지수 > 80 : 2

### 온도 라벨링?
- 냅두고 스케일링..?

### 풍속 라벨링?
- (풍속10(m/s) 기준) & (아래에 설명 더 잘 해놓음)
- 바람조금(0.0~1.5) 0
- 바람적당히(1.6~7.9) 1
- 바람많이(8.0~13.8) 2

### 미세먼지 라벨링?
- 호진오빠가 함

-------

In [1]:
import pandas as pd
import os
import dask.dataframe as dd
import progressbar

In [2]:
bar = progressbar.ProgressBar()

path_dir = 'weather/'
weatherlist = os.listdir(path_dir)

weather = pd.DataFrame()
c = []
for i in bar(weatherlist):
    k = dd.read_csv('weather/'+ i, engine = 'python', encoding = 'utf-8')
    c.append(k)
    
weather = dd.concat(c)

100% |########################################################################|


In [3]:
weather = weather.compute(scheduler = 'processes', num_workers = 4)

In [4]:
preprocess = pd.read_csv('preprocessed_final.csv')

In [5]:
from datetime import datetime
preprocess['time'] = pd.to_datetime(preprocess.time)
preprocess['time2'] = preprocess.time.dt.strftime('%H:%M')

weather['time'] = pd.to_datetime(weather.time)
weather['time2'] = weather.time.dt.strftime('%H:%M')

In [6]:
preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32956774 entries, 0 to 32956773
Data columns (total 10 columns):
Unnamed: 0           int64
parkingBikeTotCnt    float64
rackTotCnt           int64
shared               float64
stationId            object
stationLatitude      float64
stationLongitude     float64
stationName          object
time                 datetime64[ns]
time2                object
dtypes: datetime64[ns](1), float64(4), int64(2), object(3)
memory usage: 2.5+ GB


In [7]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 751374 entries, 0 to 33
Data columns (total 23 columns):
Unnamed: 0    751374 non-null int64
ID            751374 non-null int64
지점            751374 non-null object
고도            751374 non-null object
강수            751374 non-null object
강수15          751374 non-null object
강수60          751374 non-null object
강수3H          751374 non-null object
강수6H          751374 non-null object
강수12H         751374 non-null object
일강수           751374 non-null object
기온            751374 non-null object
풍향1           733410 non-null float64
풍향1_          733410 non-null object
풍속1(km/h)     732099 non-null float64
풍향10          727676 non-null float64
풍향10_         727676 non-null object
풍속10(km/h)    727724 non-null float64
습도            751373 non-null object
해면기압          751373 non-null object
위치            751373 non-null object
time          751374 non-null datetime64[ns]
time2         751374 non-null object
dtypes: datetime64[ns](1), float

In [8]:
weather[weather.습도.isna()]

,Unnamed: 0,ID,지점,고도,강수,강수15,강수60,강수3H,강수6H,강수12H,...,풍향1_,풍속1(km/h),풍향10,풍향10_,풍속10(km/h),습도,해면기압,위치,time,time2
11,12,416,은평,58m,●,0,0.5,5.5,24,44.5,...,SE,5.0,139.0,SE,12.2,NaN,NaN,NaN,2019-08-12 11:16:15,11:16


-------------------

## weather0

In [49]:
weather0 = weather.copy()

In [50]:
weather0 = weather0[['ID', '지점', '강수', '강수15', '강수60', '강수3H', '강수12H', '일강수', '기온',
                     '풍향1', '풍속1(km/h)', '풍향10', '풍속10(km/h)', '위치', 'time', 'time2']]

In [45]:
weather0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 751374 entries, 0 to 33
Data columns (total 16 columns):
ID            751374 non-null int64
지점            751374 non-null object
강수            751374 non-null object
강수15          751374 non-null object
강수60          751374 non-null object
강수3H          751374 non-null object
강수12H         751374 non-null object
일강수           751374 non-null object
기온            751374 non-null object
풍향1           733410 non-null float64
풍속1(km/h)     732099 non-null float64
풍향10          727676 non-null float64
풍속10(km/h)    727724 non-null float64
위치            751373 non-null object
time          751374 non-null datetime64[ns]
time2         751374 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(10)
memory usage: 97.5+ MB


#### 위치가 NaN인 값을 서울특별시 은평구 광진구로 바꿈. (이유가 다 있음)

In [73]:
weather0.reset_index(inplace=True)

In [21]:
weather0[weather0.ID == 416]

,ID,지점,강수,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(km/h),풍향10,풍속10(km/h),위치,time,time2
11,416,은평,○,0,0,0,0,0,35,39.0,5.8,39.4,4.7,서울특별시 은평구 진관동,2019-08-06 14:15:01,14:15
11,416,은평,○,0,0,0,0,0,27.6,176.6,10.4,128.2,8.6,서울특별시 은평구 진관동,2019-08-11 03:17:02,03:17
11,416,은평,○,0,0,0,17.5,0,24.4,0.0,0.0,0.0,0.0,서울특별시 은평구 진관동,2019-08-14 04:16:01,04:16
11,416,은평,○,0,0,0,2.5,0,24.4,251.3,2.2,184.7,1.8,서울특별시 은평구 진관동,2019-08-05 00:55:01,00:55
11,416,은평,○,0,0,0,0,0,32,87.1,2.2,70.5,5.4,서울특별시 은평구 진관동,2019-08-09 17:37:01,17:37
11,416,은평,○,0,0,0,6.0,46.5,28.0,120.1,4.0,131.0,5.8,서울특별시 은평구 진관동,2019-08-12 21:07:06,21:07
11,416,은평,○,0,0,0,0,0,26.9,0.0,0.4,0.0,0.0,서울특별시 은평구 진관동,2019-08-13 21:15:01,21:15
11,416,은평,○,0,0,0,0,0,25.6,320.3,3.2,63.3,1.8,서울특별시 은평구 진관동,2019-08-15 01:27:01,01:27
11,416,은평,○,0,0,0,0,0,29.2,149.3,2.2,151.8,2.5,서울특별시 은평구 진관동,2019-08-08 09:15:01,09:15
11,416,은평,○,0,0,0,0,0,23.5,151.9,4.7,168.9,3.2,서울특별시 은평구 진관동,2019-08-17 04:27:01,04:27


In [52]:
weather0.위치 = weather0.위치.fillna('서울특별시 은평구 진관동')

In [74]:
weather0

,index,ID,지점,강수,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(km/h),풍향10,풍속10(km/h),위치,time,time2
0,0,108,서울,○,0,0,0,0,0,36.2,40.8,5.0,28.1,7.2,서울특별시 종로구 송월동,2019-08-06 14:15:01,14:15
1,1,419,중구,○,0,0,0,0,0,34,139.3,6.5,73.5,6.8,서울특별시 중구 예장동,2019-08-06 14:15:01,14:15
2,2,415,용산 *,○,0,0,0,0,0,36.7,42.3,12.2,58.2,8.6,서울특별시 용산구 이촌동,2019-08-06 14:15:01,14:15
3,3,421,성동,○,0,0,0,0,0,34.9,8.6,8.3,39.3,9.0,서울특별시 성동구 성수동1가,2019-08-06 14:15:01,14:15
4,4,413,광진,○,0,0,0,0,0,35.6,63.5,8.6,86.3,9.0,서울특별시 광진구 자양동,2019-08-06 14:15:01,14:15
5,5,408,동대문,○,0,0,0,0,0,35.6,78.8,14.0,95.8,7.9,서울특별시 동대문구 전농동,2019-08-06 14:15:01,14:15
6,6,409,중랑,○,0,0,0,0,0,36.7,93.3,11.9,63.6,8.6,서울특별시 중랑구 면목동,2019-08-06 14:15:01,14:15
7,7,414,성북,○,0,0,0,0,0,35.6,33.3,10.8,81.8,7.2,서울특별시 성북구 정릉동,2019-08-06 14:15:01,14:15
8,8,424,강북,○,0,0,0,0,0,36.3,61.8,8.6,50.8,13.3,서울특별시 강북구 수유동,2019-08-06 14:15:01,14:15
9,9,406,도봉,○,0,0,0,0,0,36,184.1,2.5,42.7,3.6,서울특별시 도봉구 방학동,2019-08-06 14:15:01,14:15


In [56]:
weather0[weather0.위치.isna()]

,ID,지점,강수,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(km/h),풍향10,풍속10(km/h),위치,time,time2


#### 경기도 위치를 다 빼버림

In [57]:
경기도 = ['광명', '소하', '학온동', '과천', '관악(레)']
for each in 경기도:
    weather0 = weather0[weather0.지점 != each]

In [58]:
weather0.지점.unique()

array(['서울', '중구', '용산 *', '성동', '광진', '동대문', '중랑', '성북', '강북', '도봉',
       '노원', '은평', '서대문', '마포', '양천 *', '강서', '김포(공)', '구로', '금천', '한강',
       '영등포 *', '현충원', '기상청', '관악', '남현', '서초', '강남', '송파 *', '강동'],
      dtype=object)

#### 풍향1, 풍향1(km/h), 풍향10, 풍향10(km/h)  NaN 날려버리기
- 64만 데이터 중에 3만 데이터만 사라지는 거니깐 괜찮아!

In [88]:
weather0 = weather0[weather0.풍향1.notnull()]
weather0 = weather0[weather0['풍속1(km/h)'].notnull()]
weather0 = weather0[weather0.풍향10.notnull()]
weather0 = weather0[weather0['풍속10(km/h)'].notnull()]

In [89]:
weather0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623198 entries, 0 to 641060
Data columns (total 17 columns):
index         623198 non-null int64
ID            623198 non-null int64
지점            623198 non-null object
강수            623198 non-null object
강수15          623198 non-null object
강수60          623198 non-null object
강수3H          623198 non-null object
강수12H         623198 non-null object
일강수           623198 non-null object
기온            623198 non-null object
풍향1           623198 non-null float64
풍속1(km/h)     623198 non-null float64
풍향10          623198 non-null float64
풍속10(km/h)    623198 non-null float64
위치            623198 non-null object
time          623198 non-null datetime64[ns]
time2         623198 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(10)
memory usage: 85.6+ MB


-------------------

## weather 1
### 우천 라벨링
- 맑음 : 0
- 적은비(~시간당 5mm) : 1
- 많은비(>시간당 5mm ) : 2
<br><br>
- 강수 ['○', '-', '●', '.'] -> 신경쓰지 말자
- 일강수랑 강수 60만 중요한 거 같음 (시간당 자료만 필요하니깐)
- 강수60, 일강수의 object type를 float로 변경

In [93]:
weather0.drop('index', axis=1, inplace=True)

In [115]:
weather1 = weather0.drop('강수', axis=1)

In [135]:
weather1['일강수'] = weather1['일강수'].replace('.', 0)
weather1['강수60'] = weather1['강수60'].replace('.', 0)

In [136]:
weather1[weather1.강수60== '.']

,ID,지점,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(km/h),풍향10,풍속10(km/h),위치,time,time2


In [140]:
weather1["강수60"] = weather1.강수60.astype(float)
weather1["일강수"] = weather1.일강수.astype(float)

In [141]:
weather1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623198 entries, 0 to 641060
Data columns (total 15 columns):
ID            623198 non-null int64
지점            623198 non-null object
강수15          623198 non-null object
강수60          623198 non-null float64
강수3H          623198 non-null object
강수12H         623198 non-null object
일강수           623198 non-null float64
기온            623198 non-null object
풍향1           623198 non-null float64
풍속1(km/h)     623198 non-null float64
풍향10          623198 non-null float64
풍속10(km/h)    623198 non-null float64
위치            623198 non-null object
time          623198 non-null datetime64[ns]
time2         623198 non-null object
dtypes: datetime64[ns](1), float64(6), int64(1), object(7)
memory usage: 76.1+ MB


In [142]:
weather1['우천라벨링'] = weather1.강수60.apply(lambda x: 0 if x == 0 else
                                        1 if x <= 5 else 2)

#대여이력['weekend'] = 대여이력.대여일시.apply(lambda x: 0 if x.weekday() < 5 else 1)

In [144]:
weather1[weather1.강수60 > 5]

,ID,지점,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(km/h),풍향10,풍속10(km/h),위치,time,time2,우천라벨링
1140,406,도봉,1.5,6.5,23,64,58.0,25,18.0,0.7,198.3,1.8,서울특별시 도봉구 방학동,2019-08-12 10:30:02,10:30,2
1284,424,강북,0,8.0,48.5,48.5,48.5,26.8,42.8,2.5,321.2,1.8,서울특별시 강북구 수유동,2019-08-04 19:10:01,19:10,2
1285,406,도봉,0,15.0,42.5,42.5,42.5,.,328.7,2.9,322.8,1.8,서울특별시 도봉구 방학동,2019-08-04 19:10:01,19:10,2
1684,415,용산 *,2.0,7.0,9.0,9.0,9.0,24.8,54.9,6.5,42.4,7.2,서울특별시 용산구 이촌동,2019-08-15 12:31:01,12:31,2
1685,421,성동,6.0,13.0,16.0,16.0,16.0,23.8,10.7,12.6,15.3,9.7,서울특별시 성동구 성수동1가,2019-08-15 12:31:01,12:31,2
1686,413,광진,1.5,15.5,26.0,26.0,26.0,23.8,26.9,5.8,11.1,7.9,서울특별시 광진구 자양동,2019-08-15 12:31:01,12:31,2
1687,408,동대문,5.0,16.5,19.0,19.0,19.0,24.1,40.5,9.4,51.6,8.3,서울특별시 동대문구 전농동,2019-08-15 12:31:01,12:31,2
1688,409,중랑,3.5,17.5,22.5,22.5,22.5,24.4,349.1,13.0,8.9,10.1,서울특별시 중랑구 면목동,2019-08-15 12:31:01,12:31,2
1691,406,도봉,1.5,5.5,7.0,7.0,7.0,23.7,12.1,7.2,20.4,4.0,서울특별시 도봉구 방학동,2019-08-15 12:31:01,12:31,2
1692,407,노원,5.0,22.5,30.0,30.0,30.0,23.6,321.4,0.4,49.4,1.8,서울특별시 노원구 공릉동,2019-08-15 12:31:01,12:31,2


-------------------

## weather 2
### 풍속 라벨링
- km/h를 m/s로 변환
<br><br>

계급|명칭|지표면의 특징 및 풍속(m/sec)

0|고요|연기가 곧게 위로 올라감 (0.0~0.2)<br>
1|실바람|연기가 풀려서 오르고 해면은 물고기 비늘 모양의 잔 물결이 일어남 (0.3~1.5)<br>
2|남실바람|바람이 얼굴에 느껴지고 나뭇잎이 흔들리며 풍향계가 움직임 (1.6~3.3)<br>
3|산들바람|나뭇잎과 잔가지가 일정하게 흔들리고 깃발이 가볍게 휘날림 (3.4~5.4)<br>
4|건들바람|먼지가 일고 종잇조각이 날리며 나무의 잔가지가 움직임 (5.5~7.9)<br>
5|흔들바람|작은 나무 전체가 흔들리며 작은 물결이 호수에 생김 (8.0~10.7)<br>
6|된바람|큰 나뭇가지가 흔들리고 우산 쓰기가 힘들며, 큰 물결이 일기 시작함 (10.8~13.8)<br>
7|센바람|나무 전체가 흔들리고 걷기가 힘들며 해상의 물마루가 부서짐 (13.9~17.1)<br>
8|큰바람|잔가지가 꺾이고 걷기 힘들며 해상의 풍량이 높아지고 물보라가 읾 (17.2~20.7)<br>
9|큰센바람|건축물에 약간 피해가 있으며, 물보라가 소용돌이침 (20.8~24.4)<br>
10|노대바람|나무가 뽑히고 건축물에 상당한 피해가 있음 (24.5~28.4)<br>
11|왕바람|건축물에 큰 피해가 있음 (28.5~32.6)<br>
12|싹쓸바람|매우 큰 피해가 발생하며 배의 침몰이 염려됨 (32.7 이상)<br>

- 내 맘대로 정한 기준 **
- 바람조금(0.0~1.5) 0
- 바람적당히(1.6~7.9) 1
- 바람많이(8.0~13.8) 2

In [244]:
weather2 = weather1.copy()

In [245]:
weather2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623198 entries, 0 to 641060
Data columns (total 16 columns):
ID            623198 non-null int64
지점            623198 non-null object
강수15          623198 non-null object
강수60          623198 non-null float64
강수3H          623198 non-null object
강수12H         623198 non-null object
일강수           623198 non-null float64
기온            623198 non-null object
풍향1           623198 non-null float64
풍속1(km/h)     623198 non-null float64
풍향10          623198 non-null float64
풍속10(km/h)    623198 non-null float64
위치            623198 non-null object
time          623198 non-null datetime64[ns]
time2         623198 non-null object
우천라벨링         623198 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(2), object(7)
memory usage: 80.8+ MB


In [171]:
max(weather2['풍속10(km/h)'])

45.4

In [172]:
weather2.describe()

,ID,강수60,일강수,풍향1,풍속1(km/h),풍향10,풍속10(km/h),우천라벨링
count,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000
mean,413.847224,0.287312,3.345996,154.956247,5.462423,157.738625,5.479646,0.107871
std,122.581636,1.542788,9.666424,106.400965,4.493575,104.529735,4.043822,0.357565
min,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,405.000000,0.000000,0.000000,66.000000,2.200000,70.800000,2.500000,0.000000
50%,412.000000,0.000000,0.000000,140.400000,4.700000,142.800000,4.700000,0.000000
75%,419.000000,0.000000,0.000000,239.100000,7.600000,238.900000,7.600000,0.000000
max,889.000000,48.500000,72.500000,360.000000,50.800000,360.000000,45.400000,2.000000


#### 풍속10(km/h)를 풍속10(m/s)로 바꿔버림

In [173]:
def to_ms(n):
    '''input a km/hr number and convert it to m/s'''
    result = round((n * 1000)/3600, 2)
    return result

In [246]:
weather2['풍속1(km/h)'] = weather2['풍속1(km/h)'].apply(lambda x: to_ms(x))
weather2['풍속10(km/h)'] = weather2['풍속10(km/h)'].apply(lambda x: to_ms(x))

In [247]:
max(weather2['풍속10(km/h)'])

12.61

In [248]:
weather2 = weather2.rename(columns = {'풍속1(km/h)':'풍속1(m/s)'})
weather2 = weather2.rename(columns = {'풍속10(km/h)':'풍속10(m/s)'})

In [200]:
weather2.describe()

,ID,강수60,일강수,풍향1,풍속1(m/s),풍향10,풍속10(m/s),우천라벨링
count,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000
mean,413.847224,0.287312,3.345996,154.956247,1.517316,157.738625,1.522099,0.107871
std,122.581636,1.542788,9.666424,106.400965,1.248253,104.529735,1.123327,0.357565
min,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,405.000000,0.000000,0.000000,66.000000,0.610000,70.800000,0.690000,0.000000
50%,412.000000,0.000000,0.000000,140.400000,1.310000,142.800000,1.310000,0.000000
75%,419.000000,0.000000,0.000000,239.100000,2.110000,238.900000,2.110000,0.000000
max,889.000000,48.500000,72.500000,360.000000,14.110000,360.000000,12.610000,2.000000


In [257]:
weather2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623198 entries, 0 to 641060
Data columns (total 16 columns):
ID           623198 non-null int64
지점           623198 non-null object
강수15         623198 non-null object
강수60         623198 non-null float64
강수3H         623198 non-null object
강수12H        623198 non-null object
일강수          623198 non-null float64
기온           623198 non-null object
풍향1          623198 non-null float64
풍속1(m/s)     623198 non-null float64
풍향10         623198 non-null float64
풍속10(m/s)    623198 non-null float64
위치           623198 non-null object
time         623198 non-null datetime64[ns]
time2        623198 non-null object
우천라벨링        623198 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(2), object(7)
memory usage: 80.8+ MB


-------------------

## weather 3
#### 풍속 60(m/s)로 바꾸기 위해서..
- 한시간당(ex. 1:00~1:59 regardless of not having all the 59 minutes) 풍속1(m/s)의 평균을 구해서 풍속60(m/s)에 넣기

In [289]:
weather2.sort_values(by='time', ascending=False)

,ID,지점,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(m/s),풍향10,풍속10(m/s),위치,time,time2,우천라벨링,풍속60(m/s)
295872,415,용산 *,0,0.0,0,0,0.0,31,221.3,2.81,218.6,2.81,서울특별시 용산구 이촌동,2019-08-19 14:17:04,14:17,0,None
295879,406,도봉,0,0.0,0,0,0.0,30.1,342.6,1.00,341.6,1.39,서울특별시 도봉구 방학동,2019-08-19 14:17:04,14:17,0,None
295873,421,성동,0,0.0,0,0,0.0,30.6,212.6,2.31,199.1,2.81,서울특별시 성동구 성수동1가,2019-08-19 14:17:04,14:17,0,None
295874,413,광진,0,0.0,0,0,0.0,31.6,87.2,1.81,151.2,1.81,서울특별시 광진구 자양동,2019-08-19 14:17:04,14:17,0,None
295875,408,동대문,0,0.0,0,0,0.0,29.7,156.6,3.39,152.3,2.00,서울특별시 동대문구 전농동,2019-08-19 14:17:04,14:17,0,None
295876,409,중랑,0,0.0,0,0,0.0,31,67.2,2.11,124.6,2.00,서울특별시 중랑구 면목동,2019-08-19 14:17:04,14:17,0,None
295877,414,성북,0,0.0,0,0,0.0,29.8,70.1,1.69,61.3,2.00,서울특별시 성북구 정릉동,2019-08-19 14:17:04,14:17,0,None
295878,424,강북,0,0.0,0,0,0.0,30.4,145.9,2.00,102.0,1.69,서울특별시 강북구 수유동,2019-08-19 14:17:04,14:17,0,None
295880,407,노원,0,0.0,0,0,0.0,29.5,59.2,1.81,72.1,2.31,서울특별시 노원구 공릉동,2019-08-19 14:17:04,14:17,0,None
295870,108,서울,0,0.0,0,0,0.0,30.8,2.5,1.19,335.0,1.31,서울특별시 종로구 송월동,2019-08-19 14:17:04,14:17,0,None


In [293]:
k = pd.date_range(start='20190803', end='20190820', freq='H')

In [294]:
k

DatetimeIndex(['2019-08-03 00:00:00', '2019-08-03 01:00:00',
               '2019-08-03 02:00:00', '2019-08-03 03:00:00',
               '2019-08-03 04:00:00', '2019-08-03 05:00:00',
               '2019-08-03 06:00:00', '2019-08-03 07:00:00',
               '2019-08-03 08:00:00', '2019-08-03 09:00:00',
               ...
               '2019-08-19 15:00:00', '2019-08-19 16:00:00',
               '2019-08-19 17:00:00', '2019-08-19 18:00:00',
               '2019-08-19 19:00:00', '2019-08-19 20:00:00',
               '2019-08-19 21:00:00', '2019-08-19 22:00:00',
               '2019-08-19 23:00:00', '2019-08-20 00:00:00'],
              dtype='datetime64[ns]', length=409, freq='H')

In [285]:
weather2['풍속60(m/s)'] = None

In [304]:
bar = progressbar.ProgressBar()
weather3 = pd.DataFrame()
for i in bar(range(len(k)-1)):
    H_weather = weather2[(weather2.time >= k[i])&(weather2.time < k[i+1])]
    H_weather['풍속60(m/s)'] = H_weather['풍속1(m/s)'].mean()
    weather3 = pd.concat([weather3, H_weather], ignore_index=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
100% |########################################################################|


In [305]:
weather3

,ID,지점,강수15,강수60,강수3H,강수12H,일강수,기온,풍향1,풍속1(m/s),풍향10,풍속10(m/s),위치,time,time2,우천라벨링,풍속60(m/s)
0,110,김포(공),0,0.0,.,.,0.0,24.7,280.0,0.69,270.0,0.81,서울특별시 강서구 공항동,2019-08-03 01:47:02,01:47,0,0.690000
1,110,김포(공),0,0.0,.,.,0.0,24.7,280.0,0.69,270.0,0.81,서울특별시 강서구 공항동,2019-08-03 01:48:01,01:48,0,0.690000
2,110,김포(공),0,0.0,.,.,0.0,24.8,300.0,1.39,300.0,1.31,서울특별시 강서구 공항동,2019-08-03 02:06:31,02:06,0,1.097143
3,110,김포(공),0,0.0,.,.,0.0,24.8,320.0,1.00,330.0,0.50,서울특별시 강서구 공항동,2019-08-03 02:56:02,02:56,0,1.097143
4,110,김포(공),0,0.0,.,.,0.0,24.8,290.0,1.31,290.0,0.89,서울특별시 강서구 공항동,2019-08-03 02:01:02,02:01,0,1.097143
5,110,김포(공),0,0.0,.,.,0.0,24.6,290.0,1.11,310.0,1.00,서울특별시 강서구 공항동,2019-08-03 02:29:03,02:29,0,1.097143
6,110,김포(공),.,0.0,.,.,0.0,24.6,330.0,0.31,300.0,0.69,서울특별시 강서구 공항동,2019-08-03 02:37:01,02:37,0,1.097143
7,110,김포(공),0,0.0,.,.,0.0,24.8,320.0,1.00,330.0,0.50,서울특별시 강서구 공항동,2019-08-03 02:54:01,02:54,0,1.097143
8,110,김포(공),0,0.0,.,.,0.0,24.6,290.0,0.89,300.0,1.00,서울특별시 강서구 공항동,2019-08-03 02:31:01,02:31,0,1.097143
9,110,김포(공),0,0.0,.,.,0.0,24.7,310.0,1.61,310.0,1.50,서울특별시 강서구 공항동,2019-08-03 02:15:01,02:15,0,1.097143


#### 풍속 라벨링
- 바람조금(0.0~1.5) 0
- 바람적당히(1.6~7.9) 1
- 바람많이(8.0~13.8) 2

In [307]:
weather3['풍속라벨링'] = weather3['풍속60(m/s)'].apply(lambda x: 0 if (0.0 <= x and x <= 1.5)
                                                else 1 if (1.5 < x and x < 8.0)
                                                else 2)

In [309]:
weather3.describe()

,ID,강수60,일강수,풍향1,풍속1(m/s),풍향10,풍속10(m/s),우천라벨링,풍속60(m/s),풍속라벨링
count,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000,623198.000000
mean,413.847224,0.287312,3.345996,154.956247,1.517316,157.738625,1.522099,0.107871,1.517316,0.462221
std,122.581636,1.542788,9.666424,106.400965,1.248253,104.529735,1.123327,0.357565,0.735760,0.498571
min,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.395425,0.000000
25%,405.000000,0.000000,0.000000,66.000000,0.610000,70.800000,0.690000,0.000000,0.937425,0.000000
50%,412.000000,0.000000,0.000000,140.400000,1.310000,142.800000,1.310000,0.000000,1.387919,0.000000
75%,419.000000,0.000000,0.000000,239.100000,2.110000,238.900000,2.110000,0.000000,1.925564,1.000000
max,889.000000,48.500000,72.500000,360.000000,14.110000,360.000000,12.610000,2.000000,4.206794,1.000000


-------------------

In [310]:
weather2.to_csv('weather2.csv')
weather3.to_csv('weather3.csv')